In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential


In [2]:
# enter details of your AML workspace
subscription_id = "ea6d544d-2425-4667-8d38-51f050d9d69e"
resource_group = "rg-vtpoc-9dev"
workspace = "mlw-vtpoc-9dev"


In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)


In [4]:
# Define an endpoint name
endpoint_name = "my-endpoint"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(endpoint_name)

endpt-02261734577076


In [6]:
model = Model(path="./models/vt-model/model.pkl")
env = Environment(
    conda_file="../data-science/environment/train-conda.yml",
    image="crvtpoc9dev.azurecr.io/local_deploy_img:v1",
)

In [7]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../mlops/scripts", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

In [8]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (endpt-02261734577076) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-02261734577076', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/.azureml/inferencing/endpt-02261734577076'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x119f3fb50>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [10]:

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (endpt-02261734577076 / blue) .
Building Docker image from Dockerfile


Step 1/6 : FROM crvtpoc9dev.azurecr.io/local_deploy_img:v1
 ---> 847acc328956
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> f4f1617e6935
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> 2db9f7e65d67
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 66793eecef47
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 5fe4af6147fc
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> 01462078ac18
Successfully built 01462078ac18
Successfully tagged endpt-02261734577076:blue

Starting up endpoint...Done (0m 20s)


ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-02261734577076', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11a777ad0>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '3f6cff285276fcedf4231276ec74eceb', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11a79a650>, 'version': '1', 

In [12]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32806/score', 'openapi_uri': None, 'name': 'endpt-02261734577076', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11acd4ad0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [14]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=50
)

'psutil==5.9.8\r\nptyprocess==0.7.0\r\npure-eval==0.2.2\r\npyarrow==15.0.0\r\npyasn1==0.5.1\r\npyasn1-modules==0.3.0\r\npycparser==2.21\r\npydantic==1.10.14\r\npydash==7.0.5\r\nPygments==2.17.2\r\nPyJWT==2.8.0\r\npyodbc==5.1.0\r\npyparsing==3.1.1\r\npython-dateutil==2.8.2\r\npytz==2023.4\r\nPyYAML==6.0.1\r\npyzmq==25.1.2\r\nreferencing==0.33.0\r\nrequests==2.31.0\r\nrequests-oauthlib==1.3.1\r\nretrying==1.3.4\r\nrpds-py==0.18.0\r\nrsa==4.9\r\nscikit-learn==1.4.1.post1\r\nscipy==1.12.0\r\nsix==1.16.0\r\nsklearn-pandas==2.2.0\r\nsmmap==5.0.1\r\nsqlparse==0.4.4\r\nstack-data==0.6.3\r\nstrictyaml==1.7.3\r\ntenacity==8.2.3\r\nthreadpoolctl==3.3.0\r\ntornado==6.4\r\ntqdm==4.66.2\r\ntraitlets==5.14.1\r\ntyping_extensions==4.9.0\r\ntzdata==2024.1\r\nurllib3==2.2.1\r\nwcwidth==0.2.13\r\nWerkzeug==3.0.1\r\nwrapt==1.16.0\r\nxgboost==2.0.3\r\nzipp==3.17.0\r\n\r\n2024-02-26T22:35:34,972557592+00:00 | gunicorn/run | \r\n2024-02-26T22:35:35,005625467+00:00 | gunicorn/run | ###########################

In [22]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../data/test-request.json",
    local=True,
)

'{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}'